In [ ]:
## Uncomment to unzip necessary files
# !unzip proofs_miniF2F-test.zip
# !unzip utils.zip

In [ ]:
import os

PROOFS_DIRECTORY = "proofs_miniF2F-test"
check_lean_proof_WORKER_LIMIT = min(os.cpu_count(), 5)
check_proofs_in_parallel_output_arr_WORKER_LIMIT = min(os.cpu_count(), 8)
LEAN_RECURSION_DEPTH = 10_000

In [ ]:
# Install elan
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y

# Update PATH
elan_bin_path = os.path.expanduser("~/.elan/bin")
os.environ["PATH"] = elan_bin_path + ":" + os.environ["PATH"]

# Verify the installation
!lean --version
import subprocess

def setup_lean_project(project_dir="/tmp/lean_project"):
    """
    Creates a Lean project, configures it to use Mathlib,
    and downloads pre-compiled library files.
    """
    print(f"--- Setting up Lean project in: {project_dir} ---")
    os.makedirs(project_dir, exist_ok=True)

    # Content for the lakefile.lean
    lakefile_content = """
    import Lake
    open Lake DSL

    package «lean_project»

    require mathlib from git
      "https://github.com/leanprover-community/mathlib4.git"

    @[default_target]
    lean_lib «lean_project»
    """
    # Write the lakefile
    with open(os.path.join(project_dir, "lakefile.lean"), "w") as f:
        f.write(lakefile_content)

    print("--- Downloading Mathlib cache... ---")
    try:
        subprocess.run(
            ["lake", "exe", "cache", "get"],
            cwd=project_dir,
            check=True,
            capture_output=True,
            text=True
        )
        print("--- Mathlib cache downloaded successfully. ---")
    except subprocess.CalledProcessError as e:
        print("❌ Error setting up Mathlib cache.")
        print(f"--- STDOUT ---\n{e.stdout}")
        print(f"--- STDERR ---\n{e.stderr}")
        raise

    return project_dir

lean_project_path = setup_lean_project()
lean_project_path

In [ ]:
import utils
print(utils.get_proof_variants)

In [ ]:
import inspect
print(inspect.getsource(utils.get_proof_variants))

In [ ]:
import subprocess
import re
from utils import get_proof_variants
from typing import Dict
import threading
import concurrent.futures
import tempfile

LOG_PATH = os.path.expanduser("~/error.log")   # expand ~ -> /home/you/...
os.makedirs(os.path.dirname(LOG_PATH) or ".", exist_ok=True)
_log_lock = threading.Lock()

def check_lean_proof(proof_and_context: Dict, log_errors=False, max_workers=None) -> bool:
    """
    Checks a Lean‑4 proof string inside the given project using `lake`.
    If any variant succeeds, the *first* successful proof is saved to:
        corrected_proofs/<problem_id>/<proof_solver>/<attempt_id>.txt
    Returns True if a proof was saved, otherwise False.
    """
    # Verify the top‑level keys that must be present
    assert "proof" in proof_and_context, \
        "Missing 'proof' key – you need a proof string to test."
    assert "formal_statement" in proof_and_context, \
        "Missing 'formal_statement' key – you have to give the theorem statement."
    assert "project_dir" in proof_and_context, \
        "Missing 'project_dir' key – cannot locate the Lean project."
    assert "metadata" in proof_and_context, \
        "Missing 'metadata' key – you'll need context such as attempt_id."

    # Verify the required nested keys inside metadata
    assert "attempt_id" in proof_and_context["metadata"], \
        "Metadata lacks 'attempt_id' – needed to name the output file."
    assert "problem_id" in proof_and_context["metadata"], \
        "Metadata lacks 'problem_id' – needed for the directory structure."
    assert "proof_solver" in proof_and_context["metadata"], \
        "Metadata lacks 'proof_solver' – you need to know which solver produced this."

    # Unpack everything we need
    proof_string   = proof_and_context["proof"]
    statement      = proof_and_context["formal_statement"]
    project_dir    = proof_and_context["project_dir"]

    metadata       = proof_and_context["metadata"]
    attempt_id     = metadata["attempt_id"]
    problem_id     = metadata["problem_id"]
    solver_name    = metadata["proof_solver"]

    if max_workers is None:
        max_workers = check_lean_proof_WORKER_LIMIT

    # Where the successful proof will be written.
    save_dir = os.path.join(
        "corrected_proofs", problem_id, solver_name
    )
    os.makedirs(save_dir, exist_ok=True)

    # Build every candidate proof.
    proof_variants = get_proof_variants(proof_string)

    # Each variant becomes a Lean file: statement + proof.
    candidates = [
        f"{statement}\n{variant}" for variant in proof_variants
    ]

    def check_single_variant(idx, code):
        """Check a single proof variant and return (success, variant_index)"""
        # Use a temporary file with a unique name
        with tempfile.NamedTemporaryFile(mode='w', suffix='.lean', dir=project_dir, delete=False) as f:
            f.write(code)
            temp_path = f.name

        try:
            # Run Lean via lake.
            command = [
                "lake", "env", "lean",
                f"-DmaxRecDepth={LEAN_RECURSION_DEPTH}",
                os.path.basename(temp_path)  # Use basename since we're in project_dir
            ]
            result = subprocess.run(
                command,
                cwd=project_dir,
                capture_output=True,
                text=True,
                timeout=120  # 2 minutes timeout,
            )

            # Success = returncode 0 and no "error:" in stdout.
            if result.returncode == 0 and "error:" not in result.stdout:
                return (True, idx)

            if log_errors and "error:" in result.stdout:
                with _log_lock:
                    with open(LOG_PATH, "a", encoding="utf-8") as g:
                        g.writelines(result.stdout)

            return (False, idx)

        except Exception as e:
            print(f"Exception ({type(e).__name__}) for variant {idx}: {e}")
            proc = locals().get("result")
            if proc is not None:
                print("---- subprocess stdout ----")
                print(proc.stdout or "<no stdout>")
                print("---- subprocess stderr ----")
                print(proc.stderr or "<no stderr>")
            return (False, idx)
        finally:
            # Clean up the temp file
            try:
                os.remove(temp_path)
            except Exception:
                pass

    # Use ThreadPoolExecutor for parallel execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all variants for parallel execution
        future_to_index = {
            executor.submit(check_single_variant, idx, code): idx
            for idx, code in enumerate(candidates)
        }

        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_index):
            success, variant_idx = future.result()
            if success:
                # Cancel remaining tasks since we found a working proof
                for f in future_to_index:
                    f.cancel()

                # Save the successful proof
                out_path = os.path.join(save_dir, f"{attempt_id}.txt")
                with open(out_path, "w", encoding="utf-8") as out_f:
                    out_f.write(proof_variants[variant_idx])

                return True

    # No variant succeeded
    return False


# Assumptions

In [ ]:
# assert that the folder structure is as expected
from typing import Tuple

def check_structure(path: str):
    def check_numeric_children_consecutive(path: str, require_start_zero: bool = True) -> Tuple[int, int]:
        """
        Validate that:
        * `path` exists and is a directory with at least one child.
        * Immediate children are directories named in canonical natural-number form:
            "0", "1", "2", ... (no leading zeros except "0").
        * Their integer values form a consecutive range from min to max.
        * If require_start_zero is True, the range must start at 0.

        Returns:
            (min_value, max_value)

        Raises:
            AssertionError on any violation.
        """
        if not os.path.isdir(path):
            raise AssertionError(f"{path!r} is not a directory")
        entries = os.listdir(path)
        assert entries, f"{path!r} is empty"

        nat_canonical = re.compile(r"0|[1-9][0-9]*\Z")
        nums = []
        for name in entries:
            full = os.path.join(path, name)
            assert os.path.isdir(full), f"{full!r} is not a directory"
            assert nat_canonical.fullmatch(name), (
                f"{name!r} is not a canonical natural number ('0', '1', '2', ... without leading zeros)"
            )
            nums.append(int(name))

        min_n, max_n = min(nums), max(nums)
        if require_start_zero:
            assert min_n == 0, f"Sequence must start at 0 but starts at {min_n}"
        expected = set(range(min_n, max_n + 1))
        actual = set(nums)
        if actual != expected:
            missing = sorted(expected - actual)
            extra = sorted(actual - expected)
            msg = f"Immediate numeric directory names {sorted(entries)} do not form a consecutive range {min_n}..{max_n}"
            if missing:
                msg += f"; missing {missing}"
            if extra:
                msg += f"; unexpected {extra}"
            raise AssertionError(msg)
        return min_n, max_n
    # assert that it's a directory
    assert os.path.isdir(path), f"{path} is not a directory"

    # assert directory is not empty
    assert any(os.listdir(path)), f"{path} is empty"

    # assert only folders in the first level
    assert all(os.path.isdir(os.path.join(path, subdir)) for subdir in os.listdir(path)), f"Not all items in {path} are directories"

    # assert that all first level subdirectories are nats and ordered
    check_numeric_children_consecutive(path)

    # go into folder titled path/0 and find the set of folder names
    zero_folder = os.path.join(path, "0")
    assert os.path.isdir(zero_folder), f"{zero_folder} is not a directory"

    folder_names = {name for name in os.listdir(zero_folder) if os.path.isdir(os.path.join(zero_folder, name))}

    # assert that set of folder_names is the same across all number directories
    for subdir in os.listdir(path):
        subdir_path = os.path.join(path, subdir)
        if os.path.isdir(subdir_path) and subdir != "0":
            subdir_folder_names = {name for name in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, name))}
            assert subdir_folder_names == folder_names, f"Folder names in {subdir_path} do not match those in {zero_folder}"

    # assert that all subfolders have the same file names e.g. path/0/subfolder/[1..8].txt matches path/1/
    file_set = None
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for subdir in os.listdir(folder_path):
            assert os.path.isdir(os.path.join(folder_path, subdir)), f"{subdir} is not a directory"
            file_names = {name for name in os.listdir(os.path.join(folder_path, subdir)) if os.path.isfile(os.path.join(folder_path, subdir, name))}
            if file_set is None:
                file_set = file_names
            else:
                assert file_set == file_names, f"File names in {subdir} are not consistent with other subdirectories"

check_structure(PROOFS_DIRECTORY)


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
miniF2F_test_df = load_dataset("HaimingW/miniF2F-lean4", split="test").to_pandas()

In [ ]:
import json
from typing import Any, Union

def _read_text_file(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            return f.read()

In [ ]:
import json
from typing import Dict
from tqdm.auto import tqdm  # or: from tqdm import tqdm

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional, Callable

def check_proofs_in_parallel_output_arr(
    proof_contexts: List[dict],
    parallel_workers: int | None = None,
    on_progress: Optional[Callable[[int], None]] = None,
    log_errors = False
) -> List[int]:
    """
    Runs multiple proof checks in parallel (threaded).
    Returns a list of 1/0 aligned with the input order.
    Calls on_progress(1) each time an attempt completes.
    """
    # assert isinstance(parallel_workers, int) and parallel_workers > 1, "parallel_workers must be > 1"
    if not proof_contexts:
        return []

    if parallel_workers is None:
        parallel_workers = check_proofs_in_parallel_output_arr_WORKER_LIMIT

    results: List[int] = [0] * len(proof_contexts)

    def _run(idx: int, ctx: dict) -> Tuple[int, int]:
        name = f"proof_{idx}"
        ok = check_lean_proof(ctx, log_errors)
        return idx, 1 if ok else 0

    with ThreadPoolExecutor(max_workers=parallel_workers) as executor:
        futures = {executor.submit(_run, i, ctx): i for i, ctx in enumerate(proof_contexts, start=1)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                i, val = future.result()
            except Exception as exc:
                print(exc)
                i, val = idx, 0
            results[i - 1] = val
            if on_progress:
                on_progress(1)

    return results

# Evaluation Loop

In [ ]:
# Define the file to store the start index
START_IDX_FILE = "start_index.txt"

START_IDX = 0 # Default value
if os.path.exists(START_IDX_FILE):
    try:
        with open(START_IDX_FILE, "r") as f:
            content = f.read().strip()
            if content:
                START_IDX = int(content)
    except (IOError, ValueError) as e:
        print(f"Could not read {START_IDX_FILE}, defaulting to 0. Error: {e}")
        START_IDX = 0

print(f"--- 🚀 Resuming evaluation from START_IDX: {START_IDX} ---")

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import json

problem_ids = sorted([d for d in os.listdir(PROOFS_DIRECTORY) if os.path.isdir(os.path.join(PROOFS_DIRECTORY, d))], key=lambda x: int(x))
problem_ids = problem_ids[START_IDX:]
first_problem = problem_ids[0]

first_problem_path = os.path.join(PROOFS_DIRECTORY, first_problem)
model_names = sorted([d for d in os.listdir(first_problem_path) if os.path.isdir(os.path.join(first_problem_path, d))])

first_model_path = os.path.join(first_problem_path, model_names[0])
attempt_ids = sorted([os.path.splitext(f)[0] for f in os.listdir(first_model_path) if f.endswith('.txt')], key=lambda x: int(x))

num_problems = len(problem_ids)
num_models = len(model_names)
num_attempts = len(attempt_ids)

df = miniF2F_test_df
prev_success = None

with tqdm(total=num_models * num_problems,
          desc="Evaluating (problem, model) pairs",
          unit="pair",
          leave=True,
          dynamic_ncols=True) as pbar:

    for p_axis_idx, p_str in enumerate(problem_ids):
        p_axis_idx += START_IDX # Map back to the global index

        # Save progress after each problem
        try:
            with open(START_IDX_FILE, "w") as f:
                f.write(str(p_axis_idx)) # Save the current global index
        except IOError as e:
            print(f"\nWarning: Could not write to {START_IDX_FILE}. Error: {e}")

        for m_idx, model in enumerate(model_names):
            cur_desc = f"model={model} prob={p_str}"
            if prev_success is not None:
                cur_desc += f"  prev_success={prev_success}"

            pbar.set_description(cur_desc, refresh=False)

            current_proof_dir = os.path.join(PROOFS_DIRECTORY, p_str, model)
            formal_statement = miniF2F_test_df.iloc[p_axis_idx]["formal_statement"]
            header = df.iloc[p_axis_idx]["header"]

            proof_contexts = []
            for a_str in attempt_ids:
                # Read the specific attempt file.
                proof_file_path = os.path.join(current_proof_dir, f"{a_str}.txt")
                try:
                    proof_text = _read_text_file(proof_file_path)
                except FileNotFoundError:
                    proof_text = ""
                
                proof_contexts.append({
                    "formal_statement": header + "\n" + formal_statement + "\n",
                    "proof":            proof_text,
                    "project_dir":      lean_project_path,
                    "metadata": {
                        "proof_solver": model,
                        "problem_id":   p_str,
                        "attempt_id":   a_str
                    }
                })

            # Run the checks in parallel
            outcomes = check_proofs_in_parallel_output_arr(
                proof_contexts
            )

            this_success = any(outcomes)
            prev_success = "✔" if this_success else "✘"
            pbar.update(1)